### Import Libraries

In [1]:
import os
from groq import Groq
from dotenv import load_dotenv

### Load Model

In [2]:
# Load API key from .env file
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise ValueError("API KEY is missing in .env")

# Initialize Groq client
client = Groq(api_key=api_key)

# Model selection - you can change this to any other available model
# Here we use GPT-OSS 120B model through Groq API
MODEL_NAME = "openai/gpt-oss-120b"

# Define a function to generate text based on a prompt
def generate(prompt):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", # user message
             "content": prompt}
             ]
    )
    return response.choices[0].message.content

### Prompting Techniques

#### 1. Zero-Shot Prompting

A technique where the model receives only instructions, with no examples. 
It must perform the task using its general understanding.

In [3]:
# Get user input
sentence = input("Enter a sentence to translate into French: ")

# Define prompt for zero-shot prompting
prompt = f"""
Translate the input sentence from English to French.
Return only the translated sentence without any additional text in following format:

English: {sentence}
French:
"""

# Generate and print the output
print(generate(prompt))

English: I am learning AI.
French: J'apprends l'IA.


#### 2. One-Shot Prompting

A technique where the model is given one example of the desired behavior before performing the task.

In [4]:
sentence = input("Enter a sentence to summarize: ")

# Define prompt for one-shot prompting
prompt = f"""
Summarize the given sentence in a more concise way.

Example:
Input: The weather was terrible today and it ruined our plans.
Output: The bad weather spoiled our plans.

Now summarize the input sentence and provide the result in the following format:
Input: {sentence}
Output:
"""

print(generate(prompt))

Input: She studied hard and finally passed her exam.  
Output: She passed the exam after studying hard.


#### 3. Few-Shot Prompting

A prompting approach where the model is provided with multiple examples of the task. 

This helps it learn the intended style, structure, and reasoning pattern before generating the final output.

In [5]:
sentence = input("Enter a sentence to rewrite formally: ")

# Define prompt for few-shot prompting
prompt = f"""
Rewrite the given sentence in a more formal tone. Look at the examples below for guidance.

Example 1:
Input: I can't come today. I'm very busy at work.
Output: I am unable to attend today as I have multiple commitments.

Example 2:
Input: Can you finish this work quickly?
Output: Could you please complete this task at your earliest convenience?

Now rewrite the input sentence in a formal tone and provide the result in the following format:
Input: {sentence}
Output:
"""

print(generate(prompt))

Input: I apologize for the error in the file.  
Output: I extend my apologies for the error contained within the file.


#### 4. Role Prompting

A method where the model is assigned a specific identity or role (for example, tutor, analyst, expert) to control tone, perspective, and domain knowledge during the response.

In [6]:
topic = input("Enter a topic to explain: ")

# Define prompt for role prompting
prompt = f"""
You are a science tutor who explains concepts simply.
Explain the topic in short (2-3 sentences) and easy-to-understand terms.

Return the explanation in the following format:
Input: {topic}
Output:
"""

print(generate(prompt))

Input: What is photosynthesis?  
Output: Photosynthesis is the process plants use to turn sunlight, water, and carbon dioxide into food (glucose) and oxygen. It happens mainly in the leaves, where chlorophyll captures light energy to power the chemical reactions. This is how plants grow and produce the oxygen we breathe.


#### 5. Step-based Prompting

A technique instructing the model to generate the solution in a sequence of logical steps, ensuring structured and ordered reasoning.

This technique works with both short topics and simple sentences. The model follows the provided structure and expands it into a step-by-step explanation, regardless of input complexity.

In [7]:
question = input("Enter a problem or sentence you want analyzed: ")

prompt = f"""
First understand the question.
Then, identify which subject or area it relates to and analyze the key points.
Finally, provide a brief summary of your analysis. Try to be concise and clear.

Question:
{question}
"""

# Here we introduce temperature parameter to control creativity or randomness of the response, we can set between 0 (less creative) to 1 (more creative)
step_prompt_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt}
         ],
    temperature=0.5,
)

print(step_prompt_response.choices[0].message.content)

**Understanding the question**  
The query asks for the physical process by which rain is produced in the atmosphere.

**Subject/area**  
*Science – Meteorology / Atmospheric Physics (part of the water cycle).*

**Key points of analysis**  

1. **Evaporation & Transpiration** – Solar energy heats surface water (oceans, lakes, rivers) and plant leaves, turning liquid water into water‑vapour that rises into the air.  
2. **Cooling & Condensation** – As the moist air ascends, it expands and cools. When it reaches the **dew point**, water‑vapour condenses onto tiny particles (aerosols) forming microscopic droplets that become **cloud droplets**.  
3. **Cloud Development** – Continued uplift (by convection, frontal lifting, orographic lifting, etc.) adds more vapour, enlarging the cloud and increasing droplet concentration.  
4. **Droplet Growth** – Droplets grow by:  
   - **Collision‑coalescence** (larger droplets collide with and absorb smaller ones) – dominant in warm clouds.  
   - **B

#### 6. Chain-of-Thought Prompting

A prompting method where the model is explicitly asked to show its reasoning process by generating intermediate thoughts before giving the final answer.

In [8]:
question = input("Enter a question: ")

prompt = f"""
Think step by step and reason through the problem in a clear way.

Question: {question}

First explain your reasoning step by step, then give the final answer on a new line starting with: "Answer:".
"""

# Here we introduce max_tokens to control maximum length of the model response
COT_prompt_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt}
         ],
    max_tokens=500,
)

print(COT_prompt_response.choices[0].message.content)

**Step‑by‑step reasoning**

1. **Identify the known quantities**  
   - Distance traveled = 60 km  
   - Speed of the car = 30 km/h  

2. **Recall the relationship between distance, speed, and time**  
   \[
   \text{Distance} = \text{Speed} \times \text{Time}
   \]
   Solving for time gives  
   \[
   \text{Time} = \frac{\text{Distance}}{\text{Speed}}
   \]

3. **Plug the numbers into the formula**  
   \[
   \text{Time} = \frac{60\ \text{km}}{30\ \text{km/h}} = 2\ \text{hours}
   \]

4. **Interpret the result**  
   It will take the car 2 hours to travel the 60 km at a constant speed of 30 km/h.

**Answer:**  
2 hours.


#### 7. Step-back Prompting

A technique where the model is instructed to pause and look at the big picture before jumping into details. 

The model first identifies the underlying idea or structure of the problem, then uses that higher-level view to produce the answer.

In [9]:
question = input("Enter a question: ")

prompt = f"""
Take a moment to step back and look at the bigger idea behind the problem.
Start by describing the core concept that shapes the question.
Use that wider view to reach your answer.
Finish with a line beginning with: "Answer:".

Question: {question}
"""

step_back_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt}
         ],
    max_tokens=500,
    temperature=0.5
)

print(step_back_response.choices[0].message.content)

The core concept behind the question is the dynamic nature of Earth’s outer shell. The planet’s lithosphere is broken into large tectonic plates that float on the semi‑fluid asthenosphere beneath them. As these plates move—driven by mantle convection, slab pull, ridge push, and other forces—stress builds up along their boundaries and within the plates themselves. When the accumulated stress exceeds the strength of the rocks, it is released suddenly as seismic energy, causing the ground to shake. In short, earthquakes are the manifestation of the Earth constantly adjusting to the forces that drive plate motion.

Answer: Earthquakes happen because the movement of tectonic plates creates stress in the crust, and when that stress is abruptly released, it generates seismic waves that we feel as an earthquake.


#### 8. Self-Correction Prompting

A technique where the model is asked to produce an initial answer, evaluate it, and then generate a corrected or improved version.

In [10]:
question = input("Enter a question: ")

prompt = f"""
Give an initial answer to the problem.
After that, review your own answer and fix anything that feels off.
End with a line beginning with: "Final Answer:".

Question: {question}
"""

self_correction_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt}
         ],
    max_tokens=500,
    temperature=0.4
)

print(self_correction_response.choices[0].message.content)

**Initial Answer:**  
The capital of Japan is Tokyo.

**Review:**  
The answer is correct. Tokyo has been the capital of Japan since 1869 when the imperial court moved from Kyoto to Edo, which was then renamed Tokyo (meaning “Eastern Capital”). There are no alternative or disputed capitals, so the answer stands as is.

**Final Answer:** Tokyo


#### 9. Deliberate Reasoning Prompting

A structured approach that instructs the model to consider multiple possibilities or viewpoints before deciding on a final answer.

In [15]:
question = input("Enter a question: ")

prompt = f"""
Look at several possible explanations or answers before choosing one.
Write out at least two possible lines of thought.
Then decide what makes the most sense.
Finish with: "Answer:".

Question: {question}
"""

deliberate_reasoning_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt}
         ],
    max_tokens=600,
    temperature=0.5
)

print(deliberate_reasoning_response.choices[0].message.content)

**Line of thought 1 – Reproductive/Colony‑splitting reason**  
Bees (especially honeybees) swarm primarily as a natural method of colony reproduction. When a hive becomes crowded or the queen’s pheromone levels drop, a portion of the worker bees, together with the old queen, leave the original nest to find a new home. This “swarm” protects the queen and ensures that a new, genetically similar colony can be established elsewhere, allowing the species to spread and reduce competition for resources within the original hive.

**Line of thought 2 – Defense/Protection reason**  
Another possible explanation is that swarming serves as a defensive behavior. When a hive is threatened (e.g., by predators, extreme temperature, or disturbance), a large group of bees may exit together to overwhelm the threat or to relocate to a safer site. The mass departure could also be a way to confuse predators, making it harder for them to target a single bee or the queen.

**Decision**  
While swarming does h

#### 10. Directional Stimulus Prompting (DSP)

A technique where the model is given a small directional cue or micro-pattern to guide the form of reasoning, without providing a full example.

In [12]:
question = input("Enter a question: ")

prompt = f"""
Use a short guiding pattern like: value, cause, effect.
Apply that pattern to understand the problem.
Then provide your final answer on a line starting with: "Answer:".

Question: {question}
"""

DSP_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt
         }],
    max_tokens=500,
    temperature=0.4
)

print(DSP_response.choices[0].message.content)

**Pattern (value, cause, effect)**  
- **Value:** Overall physical and mental health.  
- **Cause:** Sleep delivers restorative processes (cell repair, hormone regulation, brain clearing).  
- **Effect:** Enhances cognition, memory, immune function, mood, metabolism, and long‑term disease prevention.  

**Answer:** Sleep is essential because it restores the body and brain, supporting vital functions that maintain health, cognition, and well‑being.


#### 11. ReAct Prompting

A framework where the model outputs two parts:

- **Thought**: internal reasoning
- **Action**: the final answer or operation

This separates reasoning from the final output for clearer decision-making.

In [13]:
question = input("Enter a question: ")

prompt = f"""
Write your reasoning under the label Thought.
Write your final answer under the label Action.
Keep them separate.

Question: {question}
"""

react_response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user",
         "content": prompt}
         ],
    max_tokens=500,
    temperature=0.4
)

print(react_response.choices[0].message.content)

**Thought:**  
The user wants a definition of renewable energy. I need to explain that renewable energy comes from resources that are naturally replenished on a human timescale, such as sunlight, wind, rain, tides, waves, and geothermal heat. I should mention key characteristics: sustainability, low environmental impact, and contrast with non‑renewable fossil fuels. I’ll keep the answer clear and concise.

**Action:**  
Renewable energy is energy derived from natural sources that are continuously replenished and essentially inexhaustible on a human timescale. These sources include sunlight (solar power), wind, water (hydropower, tidal and wave energy), geothermal heat from the Earth’s interior, and biomass. Because they are naturally regenerated, renewable energy resources can be used sustainably with minimal long‑term environmental impact, unlike fossil fuels which are finite and release significant greenhouse gases when burned.
